<a href="https://colab.research.google.com/github/GGpark1/Deep_Learning_Study/blob/master/CNN_practice_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#모듈 import

from tensorflow.keras.datasets import cifar10
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import cifar10

from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf

In [2]:
#시드 고정

np.random.seed(42)
tf.random.set_seed(42)

In [3]:
#데이터셋 저장

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 4s 0us/step


In [4]:
#이미지 정규화

X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [5]:
#검증 데이터 생성

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

In [6]:
#신경망 모델 구축

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu')) #필터의 갯수는 32개, 필터의 크기는 3*3, Convolution 진행, 출력함수는 ReLu, 입력값과 출력값의 크기가 같도록 padding
model.add(MaxPooling2D(2,2)) #2*2 단위로 Max Pooling 진행
model.add(Conv2D(64, (3,3))) #필터의 갯수는 54개, 크기는 3*3
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), padding='same', activation='relu')) #MaxPooling은 생략. 이미지 전체의 특징을 추출하지 않도록 Pooling을 멈춤
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [7]:
#모델 compile

model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
#모델 훈련

model.fit(X_train, y_train,
          batch_size=128,
          validation_data=(X_val, y_val),
          epochs=10)

Epoch 1/10
313/313 [==============================] - 12s 7ms/step - loss: 1.5319 - accuracy: 0.4471 - val_loss: 1.2583 - val_accuracy: 0.5579
Epoch 2/10
313/313 [==============================] - 2s 6ms/step - loss: 1.1375 - accuracy: 0.5992 - val_loss: 1.0436 - val_accuracy: 0.6355
Epoch 3/10
313/313 [==============================] - 2s 6ms/step - loss: 0.9512 - accuracy: 0.6664 - val_loss: 0.9653 - val_accuracy: 0.6613
Epoch 4/10
313/313 [==============================] - 2s 6ms/step - loss: 0.8363 - accuracy: 0.7088 - val_loss: 0.9335 - val_accuracy: 0.6727
Epoch 5/10
313/313 [==============================] - 2s 6ms/step - loss: 0.7432 - accuracy: 0.7405 - val_loss: 0.8616 - val_accuracy: 0.7043
Epoch 6/10
313/313 [==============================] - 2s 6ms/step - loss: 0.6599 - accuracy: 0.7708 - val_loss: 0.8498 - val_accuracy: 0.7119
Epoch 7/10
313/313 [==============================] - 2s 6ms/step - loss: 0.5895 - accuracy: 0.7948 - val_loss: 0.8380 - val_accuracy: 0.7163
Epoch

In [9]:
#모델 평가

model.evaluate(X_test, y_test, verbose=2)

313/313 - 1s - loss: 0.9740 - accuracy: 0.7103 - 790ms/epoch - 3ms/step


[0.9739861488342285, 0.7103000283241272]